In [ ]:
# # YOLOv5 Clone
# !git clone https://github.com/ultralytics/yolov5
# %pip install -qr requriements.txt

In [1]:
# 필요한 module upload
import torch
# yaml 파일을 만들기 위한 module
import yaml
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import cv2
# json file
import json
import os
# XML 형식의 어노테이션 파일을 활용할 예정.
import xml.etree.ElementTree as ET
# 이미지를 한 번에 긁어모으기 위한 glob 모듈
from glob import glob
# 파일 이동
import shutil
# Image를 open
from PIL import Image
# plt.imshow()후 커널 dead 현상 제거
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
# cuda 연산
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [4]:
%cd "C:\\Users\\user\\Desktop\\recognition_project\\yolov5"

C:\Users\user\Desktop\recognition_project\yolov5


In [5]:
!nvidia-smi

Tue Aug 22 19:42:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 462.75       Driver Version: 462.75       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 306... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   45C    P8    13W /  N/A |   1891MiB /  6144MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [20]:
# # 우선 file을 train과 valid, test로 나눠준다
# # directory 생성 후 나중에 상황에 맞게 분리 해준다
# path = "./VOC2028"
# img_path = path+"\\"+"JPEGImages"
# annot_path = path+"\\"+"Annotations"
# os.mkdir(img_path+"\\"+"train")
# os.mkdir(img_path+"\\"+"valid")
# os.mkdir(annot_path+"\\"+"train")
# os.mkdir(annot_path+"\\"+"valid")

In [ ]:
# yolov5 model load
# 우선은 따로 구축하진 않고 있는 parameter 활용
# parameter 활용 시 customizing 하는 방법 생각
model = torch.hub.load("ultralytics/yolov5", "yolov5s")

In [3]:
pwd

'C:\\Users\\user\\Desktop\\recognition_project'

우선 kaggle의 데이터셋에는 annotation file까지 준비가 된 상태이다.
1. 데이터 파일을 coco 형식으로 만들어 주거나 또는 이미 xml로 annotation 되어 있다.
    - 우선 이미지와 객체에 대한 label 정보를 포함하는 JSON 파일을 생성한다
    - 즉, 이미지와 객체에 대한 label 정보를 포함하는 JSON 파일을 이용하여 이미지와 라벨을 저장한다.
2. 'data.yaml' 파일 생성
    - 클래스 개수와 클래스 이름 설정
        - coco 형식으로 구성된 데이터셋에 대한 클래스 개수와 클래스 이름을 설정한다. 이 정보는 data.yaml에 포함된다
    - 학습/검증 데이터 경로 설정
        - 학습과 검증 데이터 경로를 설정. 이 경로 정보는 data.yaml 파일에 포함
    - data.yaml 파일 생성
        - 설정된 정보를 이용해 data.yaml 파일을 생성. 생성된 파일은 coco 데이터셋을 이용하는 YOLOv5 모델에서 학습에 사용된다.


다음의 아래의 코드는 annotation된 것을 YOLO에 맞게 변환 시켜준 것이다.
- COCO 형식이 아닌 PASCAL VOC형태.
- XML 형태의 ANNOTATION 파일을 YOLO 형식에 맞게

In [36]:
# YOLO 형식으로 변환할 class 이름을 지정
classes = ["hat", "person"]
# xml 파일과 이미지 파일이 저장된 폴더 경로를 지정
# 각 train, valid 별로 진행해 준다.
types = ["train", "valid"]
# yolo 형식으로 변환된 파일을 저장할 폴더를 생성
# %cd "C:\\Users\\user\\Desktop\\recognition_project\\yolov5\\VOC2028\\VOC2028"
# if not os.path.exists("labels"):
#     os.makedirs("labels")
#     %cd "C:\\Users\\user\\Desktop\\recognition_project\\yolov5\\VOC2028\\VOC2028\\labels"
#     os.makedirs("train")
#     os.makedirs("valid")
#     %cd "C:\\Users\\user\\Desktop\\recognition_project\\yolov5\\VOC2028\\VOC2028"
# xml 파일을 parsing해서 yolo 형식으로 변환
# os.listdir -> xml 데이터들이 담겨 있다.
cnt = 0
for t in types:
    xml_path = "./Annotations/" + t
    image_path = "./JPEGImages/" + t
    for xml_file_name in os.listdir(xml_path):
        # ./annotations/train\\*.xml
        xml_file_path = os.path.join(xml_path, xml_file_name)
        # xml 파일을 parsing
        tree = ET.parse(xml_file_path)
        root = tree.getroot()
        # 이미지 파일 경로 가져오기
        # image_file_name에는 해당 xml에 대응되는 이미지 파일의 이름을 나타낸다.
        image_file_name = root.find("filename").text
        if image_file_name != xml_file_name[:-4] + ".jpg":
            image_file_name = xml_file_name[:-4] + ".jpg"
        image_file_path = os.path.join(image_path, image_file_name)
        # 이미지 파일 열기
        if image_file_path[-4:] == "jpeg" or image_file_path[-4:] == "JPEG":
            image_file_path = image_file_path[:-4] + "jpg"
            print(image_file_path)
        img = Image.open(image_file_path)
        img_width, img_height = img.size # 폭 x 높이
        # YOLO 형식으로 변환 단계
        # xml_file_name[:-4] 에는 .xml 전까지의 파일 명까지만 cut
        # .txt형태로 변환
        # 각 directory별로 들어가게 된다.
        yolo_file_path = os.path.join("./labels",t,xml_file_name[:-4] + ".txt")
        with open(yolo_file_path, "w") as f:
            # 해당 xml 파일에서 <object> element를 찾아서 반복적으로 처리하는 코드.
            # <object> element는 객체의 정보를 포함하고 있다.
            # 각 객체의 정보를 추출한다. 추출된 정보를 사용하여 YOLO 형식으로 변환한 뒤, 텍스트 파일에 저장
            # 반복문을 통해 object를 하나씩 가져온다.
            for obj in root.findall("object"):
                # 각 객체 클래스의 클래스 이름을 문자열로 저장
                # class명을 하나씩 가져온다
                class_name = obj.find("name").text
                # 해당 cell의 맨 위에 classes를 정의하였다.
                # classes.index(class_name) 이라고 하면 내가 정의한 list의 순서대로 index를 반환한다.
                if class_name not in classes:
                    continue
                class_id = classes.index(class_name)

                # 해당 <object> element에서 하위 element인 bndbox element를 찾아서 반환한다.
                bbox = obj.find("bndbox")
                x_min = int(bbox.find("xmin").text)
                y_min = int(bbox.find("ymin").text)
                x_max = int(bbox.find("xmax").text)
                y_max = int(bbox.find("ymax").text)

                # normalize 시켜주는 과정
                # 1.0은 이미지의 너비 또는 높이를 1로 정규화 하는데 사용
                # size[0]: 폭
                dx = int(img.size[0])
                # size[1]: 높이
                dy = int(img.size[1])
                # x_center는 해당 객체의 bounding box 중심점의 x좌표
                # 값을 YOLO 형식으로 bounding box 좌표를 변환하기 위해 사용
                # 0 ~ 1 사이로 정규화 되어 있으므로 이 값을 YOLO 형식으로 변환 시에도 0 ~ 1
                # y_center도 마찬가지의 원리
                # x_center, y_center, w, h는 YOLO 형식으로 bounding box 좌표를 나타내는 값
                # YOLO: 이미지를 Grid로 나누고 각 grid 셀마다 bounding box를 예측
                # bounding box는 해당 셀에 대해 상대적인 위치와 크기로 표현
                # 이를 위해 x_center, y_center, w, h를 사용한다
                # x_center: bounding box 중심점의 x 좌표
                x_center = ((x_min+x_max) / 2) / dx
                # y_center: bounding box의 중심점의 y좌표
                y_center = ((y_min+y_max) / 2) / dy
                # bounding box의 너비(해당 셀의 너비를 기준으로 상대적인 크기)
                w = (x_max - x_min) / img_width
                # bounding box의 높이(해당 셀의 높이를 기준으로 한 상대적인 크기)
                h = (y_max - y_min) / img_height

                # YOLO 형식으로 변환된 Bounding box 정보를 txt 파일에 기록
                # class_id: 해당 객체의 class id를 나타낸다.
                # classes에 설정된 class 명의 index를 할당
                # helmet, head, person의 클래스 ID를 할당
                # x_center, y_center, w, h는 YOLO 형식으로 bounding box 좌표를 나타낸다.
                f.write(f"{class_id} {x_center:.5f} {y_center:.5f} {w:.5f} {h:.5f}\n")
            print("Convert these files, {} file and {} file, into the YOLO format".format(xml_file_name, image_file_name))

Convert these files, 000000.xml file and 000000.jpg file, into the YOLO format
Convert these files, 000002.xml file and 000002.jpg file, into the YOLO format
Convert these files, 000003.xml file and 000003.jpg file, into the YOLO format
Convert these files, 000004.xml file and 000004.jpg file, into the YOLO format
Convert these files, 000005.xml file and 000005.jpg file, into the YOLO format
Convert these files, 000006.xml file and 000006.jpg file, into the YOLO format
Convert these files, 000007.xml file and 000007.jpg file, into the YOLO format
Convert these files, 000008.xml file and 000008.jpg file, into the YOLO format
Convert these files, 000009.xml file and 000009.jpg file, into the YOLO format
Convert these files, 000010.xml file and 000010.jpg file, into the YOLO format
Convert these files, 000011.xml file and 000011.jpg file, into the YOLO format
Convert these files, 000012.xml file and 000012.jpg file, into the YOLO format
Convert these files, 000013.xml file and 000013.jpg 

Convert these files, 000380.xml file and 000380.jpg file, into the YOLO format
Convert these files, 000381.xml file and 000381.jpg file, into the YOLO format
Convert these files, 000382.xml file and 000382.jpg file, into the YOLO format
Convert these files, 000383.xml file and 000383.jpg file, into the YOLO format
Convert these files, 000384.xml file and 000384.jpg file, into the YOLO format
Convert these files, 000385.xml file and 000385.jpg file, into the YOLO format
Convert these files, 000386.xml file and 000386.jpg file, into the YOLO format
Convert these files, 000387.xml file and 000387.jpg file, into the YOLO format
Convert these files, 000388.xml file and 000388.jpg file, into the YOLO format
Convert these files, 000389.xml file and 000389.jpg file, into the YOLO format
Convert these files, 000390.xml file and 000390.jpg file, into the YOLO format
Convert these files, 000391.xml file and 000391.jpg file, into the YOLO format
Convert these files, 000392.xml file and 000392.jpg 

Convert these files, 000760.xml file and 000760.jpg file, into the YOLO format
Convert these files, 000761.xml file and 000761.jpg file, into the YOLO format
Convert these files, 000762.xml file and 000762.jpg file, into the YOLO format
Convert these files, 000763.xml file and 000763.jpg file, into the YOLO format
Convert these files, 000764.xml file and 000764.jpg file, into the YOLO format
Convert these files, 000765.xml file and 000765.jpg file, into the YOLO format
Convert these files, 000766.xml file and 000766.jpg file, into the YOLO format
Convert these files, 000767.xml file and 000767.jpg file, into the YOLO format
Convert these files, 000768.xml file and 000768.jpg file, into the YOLO format
Convert these files, 000769.xml file and 000769.jpg file, into the YOLO format
Convert these files, 000770.xml file and 000770.jpg file, into the YOLO format
Convert these files, 000771.xml file and 000771.jpg file, into the YOLO format
Convert these files, 000772.xml file and 000772.jpg 

Convert these files, 001153.xml file and 001153.jpg file, into the YOLO format
Convert these files, 001154.xml file and 001154.jpg file, into the YOLO format
Convert these files, 001155.xml file and 001155.jpg file, into the YOLO format
Convert these files, 001156.xml file and 001156.jpg file, into the YOLO format
Convert these files, 001157.xml file and 001157.jpg file, into the YOLO format
Convert these files, 001158.xml file and 001158.jpg file, into the YOLO format
Convert these files, 001159.xml file and 001159.jpg file, into the YOLO format
Convert these files, 001160.xml file and 001160.jpg file, into the YOLO format
Convert these files, 001161.xml file and 001161.jpg file, into the YOLO format
Convert these files, 001162.xml file and 001162.jpg file, into the YOLO format
Convert these files, 001163.xml file and 001163.jpg file, into the YOLO format
Convert these files, 001164.xml file and 001164.jpg file, into the YOLO format
Convert these files, 001165.xml file and 001165.jpg 

Convert these files, 001561.xml file and 001561.jpg file, into the YOLO format
Convert these files, 001562.xml file and 001562.jpg file, into the YOLO format
Convert these files, 001563.xml file and 001563.jpg file, into the YOLO format
Convert these files, 001564.xml file and 001564.jpg file, into the YOLO format
Convert these files, 001565.xml file and 001565.jpg file, into the YOLO format
Convert these files, 001566.xml file and 001566.jpg file, into the YOLO format
Convert these files, 001567.xml file and 001567.jpg file, into the YOLO format
Convert these files, 001568.xml file and 001568.jpg file, into the YOLO format
Convert these files, 001569.xml file and 001569.jpg file, into the YOLO format
Convert these files, 001570.xml file and 001570.jpg file, into the YOLO format
Convert these files, 001571.xml file and 001571.jpg file, into the YOLO format
Convert these files, 001572.xml file and 001572.jpg file, into the YOLO format
Convert these files, 001573.xml file and 001573.jpg 

...

Convert these files, PartB_01887.xml file and PartB_01887.jpg file, into the YOLO format
Convert these files, PartB_01888.xml file and PartB_01888.jpg file, into the YOLO format
Convert these files, PartB_01890.xml file and PartB_01890.jpg file, into the YOLO format
Convert these files, PartB_01891.xml file and PartB_01891.jpg file, into the YOLO format
Convert these files, PartB_01892.xml file and PartB_01892.jpg file, into the YOLO format
Convert these files, PartB_01893.xml file and PartB_01893.jpg file, into the YOLO format
Convert these files, PartB_01894.xml file and PartB_01894.jpg file, into the YOLO format
Convert these files, PartB_01895.xml file and PartB_01895.jpg file, into the YOLO format
Convert these files, PartB_01897.xml file and PartB_01897.jpg file, into the YOLO format
Convert these files, PartB_01898.xml file and PartB_01898.jpg file, into the YOLO format
Convert these files, PartB_01899.xml file and PartB_01899.jpg file, into the YOLO format
Convert these files, 

Convert these files, PartB_02009.xml file and PartB_02009.jpg file, into the YOLO format
Convert these files, PartB_02010.xml file and PartB_02010.jpg file, into the YOLO format
Convert these files, PartB_02011.xml file and PartB_02011.jpg file, into the YOLO format
Convert these files, PartB_02012.xml file and PartB_02012.jpg file, into the YOLO format
Convert these files, PartB_02013.xml file and PartB_02013.jpg file, into the YOLO format
Convert these files, PartB_02014.xml file and PartB_02014.jpg file, into the YOLO format
Convert these files, PartB_02015.xml file and PartB_02015.jpg file, into the YOLO format
Convert these files, PartB_02016.xml file and PartB_02016.jpg file, into the YOLO format
Convert these files, PartB_02017.xml file and PartB_02017.jpg file, into the YOLO format
Convert these files, PartB_02018.xml file and PartB_02018.jpg file, into the YOLO format
Convert these files, PartB_02019.xml file and PartB_02019.jpg file, into the YOLO format
Convert these files, 

Convert these files, PartB_02127.xml file and PartB_02127.jpg file, into the YOLO format
Convert these files, PartB_02128.xml file and PartB_02128.jpg file, into the YOLO format
Convert these files, PartB_02129.xml file and PartB_02129.jpg file, into the YOLO format
Convert these files, PartB_02130.xml file and PartB_02130.jpg file, into the YOLO format
Convert these files, PartB_02131.xml file and PartB_02131.jpg file, into the YOLO format
Convert these files, PartB_02132.xml file and PartB_02132.jpg file, into the YOLO format
Convert these files, PartB_02133.xml file and PartB_02133.jpg file, into the YOLO format
Convert these files, PartB_02134.xml file and PartB_02134.jpg file, into the YOLO format
Convert these files, PartB_02135.xml file and PartB_02135.jpg file, into the YOLO format
Convert these files, PartB_02136.xml file and PartB_02136.jpg file, into the YOLO format
Convert these files, PartB_02137.xml file and PartB_02137.jpg file, into the YOLO format
Convert these files, 

Convert these files, PartB_02245.xml file and PartB_02245.jpg file, into the YOLO format
Convert these files, PartB_02246.xml file and PartB_02246.jpg file, into the YOLO format
Convert these files, PartB_02247.xml file and PartB_02247.jpg file, into the YOLO format
Convert these files, PartB_02248.xml file and PartB_02248.jpg file, into the YOLO format
Convert these files, PartB_02249.xml file and PartB_02249.jpg file, into the YOLO format
Convert these files, PartB_02250.xml file and PartB_02250.jpg file, into the YOLO format
Convert these files, PartB_02251.xml file and PartB_02251.jpg file, into the YOLO format
Convert these files, PartB_02252.xml file and PartB_02252.jpg file, into the YOLO format
Convert these files, PartB_02253.xml file and PartB_02253.jpg file, into the YOLO format
Convert these files, PartB_02254.xml file and PartB_02254.jpg file, into the YOLO format
Convert these files, PartB_02255.xml file and PartB_02255.jpg file, into the YOLO format
Convert these files, 

Convert these files, PartB_02361.xml file and PartB_02361.jpg file, into the YOLO format
Convert these files, PartB_02362.xml file and PartB_02362.jpg file, into the YOLO format
Convert these files, PartB_02363.xml file and PartB_02363.jpg file, into the YOLO format
Convert these files, PartB_02364.xml file and PartB_02364.jpg file, into the YOLO format
Convert these files, PartB_02365.xml file and PartB_02365.jpg file, into the YOLO format
Convert these files, PartB_02366.xml file and PartB_02366.jpg file, into the YOLO format
Convert these files, PartB_02367.xml file and PartB_02367.jpg file, into the YOLO format
Convert these files, PartB_02368.xml file and PartB_02368.jpg file, into the YOLO format
Convert these files, PartB_02369.xml file and PartB_02369.jpg file, into the YOLO format
Convert these files, PartB_02370.xml file and PartB_02370.jpg file, into the YOLO format
Convert these files, PartB_02371.xml file and PartB_02371.jpg file, into the YOLO format
Convert these files, 

In [14]:
# yaml 파일 제작
# train, test, valid, number of classes, classes name 지정
%cd "C:\\Users\\user\\Desktop\\recognition_project\\yolov5\\VOC2028"
data = {'train': "./VOC2028/VOC2028/images/train",
        "val": "./VOC2028/VOC2028/images/valid",
        "nc": '2',
        "names": ['hat', 'person']}

# data.yaml은 위의 dictionary 변수 명과 같이 한다.
with open("data.yaml", "w") as f:
    yaml.dump(data, f)

C:\Users\user\Desktop\recognition_project\yolov5\VOC2028


In [16]:
%cd "C:\\Users\\user\\Desktop\\recognition_project\\yolov5"

C:\Users\user\Desktop\recognition_project\yolov5


In [39]:
%cd ..

C:\Users\user\Desktop\recognition_project\yolov5


In [ ]:
torch.cuda.init()

YOLOv5 사용 결과
1. yolov5s나 m이나 큰 차이는 없다.
2. batch size 늘리고 epoch 수를 늘리려면 yolov5s를 사용하는 게 맞다.
3. epoch 수는 최소 100 epoch을 진행해보고 hyperparameter를 조정하는 등의 과정을 거쳐야 할 듯 싶다.

In [18]:
# 추후에 프로젝트 시작 후에 train, validation, test 진행
# --conf 옵션은 confidence(신뢰도) 임계값을 설정하는 옵션이다.
# batch 64 이상은 학습 중에 GPU 메모리 부족을 야기할 수 있다.
!python train.py --img 640 --batch 4 --epochs 50 --data ./VOC2028/data.yaml --weights ./runs/train/exp7/weights/best.pt  --device cuda:0

^C


- Data Augmentation 진행
- Fine Tuning 진행

In [6]:
!python detect.py --source "./archive/images/test" --weights "./runs/train/exp2/weights/last.pt" --device cuda:0

detect: weights=['./runs/train/exp2/weights/last.pt'], source=./archive/images/test, data=data\coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=cuda:0, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  v7.0-133-gcca5e21 Python-3.9.13 torch-1.13.1+cu116 CUDA:0 (GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
YOLOv5l summary: 267 layers, 46119048 parameters, 0 gradients, 107.7 GFLOPs
Traceback (most recent call last):
  File "C:\Users\user\Desktop\recognition_project\yolov5\detect.py", line 270, in <module>
    main(opt)
  File "C:\Users\user\Desktop\recognition_project\yolov5\detect.py", line 265, in main
    run(**vars(opt))
  File "C:\Users\user\anaconda3\lib\site-packages\torch\autograd\g

70 epoch 학습 결과
- 헬멧을 쓴 사람이면 helmet으로 예측
- 헬멧이 아닌 사람이면 head로 예측

In [15]:
!python detect.py --source "./data/sample_data/" --weights "./runs/train/exp7/weights/last.pt" --device cuda:0 --save-crop --save-txt

detect: weights=['./runs/train/exp7/weights/last.pt'], source=./data/sample_data/, data=data\coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=cuda:0, view_img=False, save_txt=True, save_conf=False, save_crop=True, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  v7.0-133-gcca5e21 Python-3.9.13 torch-1.13.1+cu116 CUDA:0 (GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
YOLOv5l summary: 267 layers, 46119048 parameters, 0 gradients, 107.7 GFLOPs
image 1/2 C:\Users\user\Desktop\recognition_project\yolov5\data\sample_data\KakaoTalk_20230906_180327417.jpg: 640x480 1 head, 20.5ms
video 2/2 (1/132) C:\Users\user\Desktop\recognition_project\yolov5\data\sample_data\check.mp4: 640x384 1 helmet, 17.4ms
video 2/2 (2/132) C:\Users\user\Desktop\recognition_project\yolov

In [20]:
!python3 detect.py [--save-crops]

Python


In [ ]:
Image.open("./runs/detect/exp"+"\\"+picts_2[386])

In [ ]:
Image.open("./runs/detect/exp"+"\\"+picts_2[756])

In [ ]:
pwd

In [8]:
!nvidia-smi

Thu Sep  7 02:04:57 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 462.75       Driver Version: 462.75       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 306... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   47C    P8    13W /  N/A |    121MiB /  6144MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------